RN _G 효윻화  
label

In [1]:
import pandas as pd
import tensorflow as tf
import itertools
import numpy as np
from tqdm import tqdm
# import helpers
import warnings
import pickle
import random
import os
from tensorflow.contrib import slim
from tensorflow.contrib import rnn  #import LSTMCell, LSTMStateTupleTuple
warnings.filterwarnings('ignore')

In [2]:
with open('babi_preprocessed/train_dataset.pkl', 'rb') as f:
    train = pickle.load(f)

with open('babi_preprocessed/test_dataset.pkl', 'rb') as f:
    test = pickle.load(f)
    
# train = (Question, Answer, Context, Labels, Q_length, C_seq_length)

# max_ = 0
# for i in train[2]:
#     for j in i:
#         if len(j) > max_:
#             max_ = len(i)
# print(max_)

In [3]:
n_hidden = 32
n_classes = 159

q_vocab = 88
q_max_word = 12

c_vocal = 124
c_seq_num = 20
c_max_word = 12

batch_size = 64

question = tf.placeholder(tf.float32, [batch_size, q_max_word ,q_vocab]) # [batch_size, num_word, one_hot dim] 
q_length= tf.placeholder(tf.int32, [batch_size])

context = tf.placeholder(tf.float32, [batch_size, c_seq_num, c_max_word, c_vocal]) 
                                                #  [batch_size, num_support_sentence, num_word, one_hot dim]
c_seq_length = tf.placeholder(tf.int32, [batch_size,c_seq_num])
# c_num_seq = tf.placeholder(tf.int32, [None])

label = tf.placeholder(tf.int32, [batch_size,c_seq_num, c_seq_num])

answer = tf.placeholder(tf.int32, [batch_size, n_classes])

In [4]:
def question_LSTM(q, seqlen):
    q = tf.unstack(q, q_max_word,1)
    lstm_cell = rnn.LSTMCell(n_hidden)
    outputs, states = rnn.static_rnn(lstm_cell, q, dtype = tf.float32, sequence_length=seqlen)
    
    # change dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1,0,2])
    
#     # 색인생성을 위함, 나중에 label의 순서정보로 바꿔야하나?
#     # aymericdamien
    batchsize = tf.shape(outputs)[0]
    index = tf.range(0, batchsize) * q_max_word + (seqlen -1)
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)
    return outputs

In [5]:
def context_LSTM_helper(q, seqlen, reuse):
    with tf.variable_scope("cLSTM_helper", reuse=reuse) as scope:
        q = tf.unstack(q, c_max_word,1)
        lstm_cell = rnn.LSTMCell(n_hidden)
        outputs, states = rnn.static_rnn(lstm_cell, q, dtype = tf.float32, sequence_length=seqlen)

        # change dimension to [batch_size, n_step, n_input]
        outputs = tf.stack(outputs)
        outputs = tf.transpose(outputs, [1,0,2])

        batchsize = tf.shape(outputs)[0]
        index = tf.range(0, batchsize) * c_max_word + (seqlen -1)
        outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)
        return outputs    

def context_LSTM(c, c_seqlen):
    result = []
    reuse = False
    for i in range(batch_size):
        sentences, sent_lengths = c[i], c_seqlen[i]
        tmp_out = context_LSTM_helper(sentences, sent_lengths,reuse)
        
        if not reuse:
            reuse = True
        
        result.append(tmp_out)
    return tf.stack(result)

In [6]:
def combination_helper(objects):
    list_ = []
    for i in range(c_seq_num):
        list_.append(objects[i])
    return itertools.combinations(list_,2)

def combination(set_objects, questions, batch_size = batch_size):
    result = []
    for i in range(batch_size):
        for (object_i, object_j) in combination_helper(set_objects[i]):
            result.append(tf.concat([object_i, object_j, questions[i]], axis = 0))
    return tf.stack(result)

In [7]:
def RN_g(concat, scope = "g_theta"):
    # make object pair with question using tf.concat
#     opwq = tf.reshape(tf.concat([object_i, object_j, question], axis = 0), [1,-1])
    net = slim.fully_connected(concat, 256 ) #reuse=reuse,  scope="g_1")
    net = slim.fully_connected(net, 256) #, scope="g_2")
    net = slim.fully_connected(net, 256) #, scope="g_3")
    net = slim.fully_connected(net, 256) #, scope="g_4")
    return net

In [8]:
def RN_f(element_sum, scope= "f_phi"):
    net = slim.fully_connected(element_sum, 256, scope="f_1")
    net = slim.fully_connected(net, 512, scope="f_2")
    net = slim.fully_connected(net, 159, scope="f_3")
    return net

In [9]:
def batch_load(data, batch_size):
    # data = (Question, Answer, Context, Labels, Q_length, C_seq_length)
    randix = np.random.randint(len(data[0]), size = batch_size)
    batch_q = np.array(data[0])[randix]
    batch_a = np.array(data[1])[randix]
    batch_c = data[2][randix]
    batch_l = np.array(data[3])[randix]
    batch_ql = np.array(data[4])[randix]
    batch_cl = np.array(data[5])[randix]
    return batch_q, batch_a, batch_c, batch_l, batch_ql, batch_cl

# label에 따른 순서 지정  
# context의 문장을 무조건 20개 사용(비효율적)->개선 할 수 있는 구현방법 

In [10]:
import time

In [11]:
t1 = time.time()

objects_units = context_LSTM(context, c_seq_length)
q_unit = question_LSTM(question, q_length)

MLP_g = RN_g(combination(objects_units, q_unit))

merged = []
for i in range(batch_size):
    merged.append(tf.reduce_sum(MLP_g[i*batch_size:(i+1)*batch_size],axis=0))
merged = tf.stack(merged)

out = RN_f(merged)

t2 = time.time()

In [12]:
print(t2-t1)

86.97003149986267


In [13]:
t3 = time.time()

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out, labels = answer ))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(cost)
correct_pred = tf.equal(tf.argmax(out,1), tf.argmax(answer,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

t4 = time.time()

In [14]:
print(t4-t3)

222.3263988494873


뭐지 이거.. 왜 이따위로... loss구하는데 4분이나 걸리네.. 뭘까요이건..

# training session

In [15]:
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True,gpu_options=gpu_options)

if not os.path.exists('model/'):
    os.makedirs('model/')

with tf.Session(config= config) as sess:
    sess.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver(max_to_keep=4)
    
    for i in range(1,10001):
        batch_q, batch_a, batch_c, batch_l, batch_ql, batch_cl = batch_load(train, batch_size)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict = {question: batch_q, q_length: batch_ql, context: batch_c, c_seq_length: batch_cl, answer : batch_a})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        optimizer.run(feed_dict = {question: batch_q, q_length: batch_ql, context: batch_c, c_seq_length: batch_cl, answer : batch_a})
        
        if i % 2500 == 0:
            saver.save(sess, save_path = 'model/rn', global_step = i)
        
    print("learning finished!")
    batch_q, batch_a, batch_c, batch_l, batch_ql, batch_cl = batch_load(test, batch_size)
    print('test accuracy %g' % accuracy.eval(feed_dict = {question: batch_q, q_length: batch_ql, context: batch_c, c_seq_length: batch_cl, answer : batch_a}))

step 100, training accuracy 0.203125
step 200, training accuracy 0.109375
step 300, training accuracy 0.140625
step 400, training accuracy 0.125
step 500, training accuracy 0.140625
step 600, training accuracy 0.171875
step 700, training accuracy 0.09375
step 800, training accuracy 0.125
step 900, training accuracy 0.140625
step 1000, training accuracy 0.125
step 1100, training accuracy 0.0625
step 1200, training accuracy 0.109375
step 1300, training accuracy 0.171875
step 1400, training accuracy 0.15625
step 1500, training accuracy 0.140625
step 1600, training accuracy 0.15625
step 1700, training accuracy 0.0625
step 1800, training accuracy 0.09375
step 1900, training accuracy 0.078125
step 2000, training accuracy 0.109375
step 2100, training accuracy 0.0625
step 2200, training accuracy 0.171875
step 2300, training accuracy 0.09375
step 2400, training accuracy 0.140625
step 2500, training accuracy 0.078125
step 2600, training accuracy 0.09375
step 2700, training accuracy 0.203125
step

# model load

In [ ]:
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    saver.restore(sess, 'model/fc-1')
    # training cycle
    for epoch in range(3):
        avg_loss = 0.
        n_iters_per_epoch = int(mnist.train.num_examples / batch_size)
        # loop over all batches
        for i in range(n_iters_per_epoch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            # run optimization op (backprop) and loss op (to get loss value)
            _, c = sess.run([optimizer, loss], feed_dict={x: x_batch, y: y_batch})
            # compute average loss
            avg_loss += c / n_iters_per_epoch
        print "Epoch %d, Loss: %.3f"% (epoch+1, avg_loss)
        if epoch % save_every == 0:
            saver.save(sess, save_path='model/fc', global_step=epoch+1)
            
    print "Finished training!"
    print "\nTest accuracy:", sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels})